# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

In [1]:
# технический блок со всем необходимым
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Нулевое задание

Исследуйте таблицы — выведите первые строки.

In [2]:
def table_test(table):
    query = '''
    SELECT *
    FROM {0}
    LIMIT 1
    '''.format(table)
    print('Пример таблицы {0}'.format(table))
    display(pd.io.sql.read_sql(query, con = engine))
    print('='*100)

for table_name in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    table_test(table_name)

Пример таблицы books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


Пример таблицы authors


,author_id,author
0,1,A.S. Byatt


Пример таблицы publishers


,publisher_id,publisher
0,1,Ace


Пример таблицы ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


Пример таблицы reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


В целом, проблем по талицам не вижу, типы данных, заявленные в схеме соответствуют наполнению таблиц. Можно приступать к заданиям.

## Первое задание

Посчитайте, сколько книг вышло после 1 января 2000 года.

In [3]:
query = '''
SELECT COUNT(*) AS book_count
FROM books
WHERE publication_date >= '2000-01-01'
'''
display(pd.io.sql.read_sql(query, con = engine))

,book_count
0,821


После 1 января 2000 года была выпущена 821 книга.

## Второе задание

Для каждой книги посчитайте количество обзоров и среднюю оценку. Дату публикации добавлю на случай, если были книги с одинаковыми названиями выходили в разные года.

Чтобы потом себя проверить, выведем общее количество обзоров.

In [5]:
query = '''
SELECT COUNT(*)
FROM reviews
'''
display(pd.io.sql.read_sql(query, con = engine))

,count
0,2793


In [7]:
query = '''
WITH review_counter AS
    (SELECT book_id,
            COUNT(*) AS review_count
    FROM reviews
    GROUP BY book_id),

rating_meaner AS
    (SELECT book_id,
            AVG(rating) AS avg_rating
    FROM ratings
    GROUP BY book_id)

SELECT b.title,
       b.publication_date,
       rc.review_count,
       rm.avg_rating
FROM books AS b
JOIN review_counter AS rc ON b.book_id = rc.book_id
JOIN rating_meaner AS rm ON b.book_id = rm.book_id
'''
output = pd.io.sql.read_sql(query, con = engine)
display(output)

,title,publication_date,review_count,avg_rating
0,The Body in the Library (Miss Marple #3),2006-09-01,2,4.500000
1,Galápagos,1999-01-12,2,4.500000
2,A Tree Grows in Brooklyn,2006-05-30,5,4.250000
3,Undaunted Courage: The Pioneering First Missio...,2003-10-06,2,4.000000
4,The Prophet,2010-01-01,4,4.285714
...,...,...,...,...
989,Alice in Wonderland,2004-09-07,4,4.230769
990,A Woman of Substance (Emma Harte Saga #1),2005-12-01,2,5.000000
991,Christine,2001-06-13,3,3.428571
992,The Magicians' Guild (Black Magician Trilogy #1),2004-01-31,2,3.500000


In [8]:
sum(output['review_count'])

2793

Сделано. Просто ради интереса посмотрим на книгу с самым большим количеством отзывов и лучшим рейтингом, на случай если таких книг несколько, и наоборот: книгу с лучшим рейтином и самым больши количеством отзывов.

In [9]:
display(output.sort_values(['review_count', 'avg_rating']).tail(1))
display(output.sort_values(['avg_rating', 'review_count']).tail(1))

,title,publication_date,review_count,avg_rating
161,Twilight (Twilight #1),2006-09-06,7,3.6625


,title,publication_date,review_count,avg_rating
911,A Dirty Job (Grim Reaper #1),2006-03-21,4,5.0


Первая часть Сумерек собрала больше всех отзывов, но средний рейтинг у неё не очень высокий. А вот книгу A Dirty Job (Grim Reaper #1) я, пожалуй, добавлю в список чтения на лето :)

## Третье задание

Определите издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [10]:
query = '''
SELECT p.publisher,
       COUNT(b.*) AS book_count
FROM books AS b
JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY book_count DESC
LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine))

,publisher,book_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Хотелось здесь написать список знаковых для этого издательства Penguin Books книг, но всё оказалось намного интересней - данное издательство является знаковым для Великобритании, поскольку ещё до Второй мировой войны популяризовало книги в мягкой обложке для людей всей страны в довольно тяжёлый преиод истории.

## Четвёртое задание

Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [11]:
query = '''
WITH top_books AS 
    (SELECT b.*,
            COUNT(rat.*) as rating_count
     FROM books AS b
     JOIN ratings AS rat ON b.book_id = rat.book_id
     GROUP BY b.book_id
     HAVING COUNT(rat.*) >= 50)
     
SELECT a.author,
       AVG(rat.rating) AS avg_rating
FROM top_books AS b
JOIN authors AS a ON b.author_id = a.author_id
JOIN ratings AS rat ON b.book_id = rat.book_id
GROUP BY a.author
ORDER BY avg_rating DESC
LIMIT 5
'''
display(pd.io.sql.read_sql(query, con = engine))

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


Здесь победителем оказалась пара автора Дж. К. Роулинг и иллюстратора Мэри Грандпре, которые вместе работали над серией книг о Гарри Поттере. С учётом этого становится неудивительно, что у них такой высокий средний рейтинг.

## Пятое задание

Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
query = '''
WITH top_users AS 
    (SELECT username,
            COUNT(DISTINCT rating_id) as rating_count
    FROM ratings
    GROUP BY username
    HAVING COUNT(DISTINCT rating_id) >= 50),
     
review_counter AS
    (SELECT u.username,
            COUNT(DISTINCT review_id) as review_count
     FROM top_users AS u
     JOIN reviews AS rev ON u.username = rev.username
     GROUP BY u.username)

SELECT AVG(review_count)
FROM review_counter
'''
display(round(pd.io.sql.read_sql(query, con = engine), 0))

,avg
0,24.0


В среднем, пользователи поставившие больше 50 оценок, также пишут довольно много отзывов. Вполне логично, если стимулировать пользователей ставить оценки и предлагать при этом писать отзывы, то довольно много людей хоть что-то да напишут.